В данной лекции мы с вами научимся пользоваться Keras'ом — частью фреймворка TensorFlow. Keras предназначен для упрощения разработки моделей для решения повседневных задач. В случае исследовательских работ или построения сложных схем лучше делать это на уровне TensorFlow — т.е. пользуясь тензорами, и самостоятельно определяя слои, и как они будут соединяться. Keras же помогает быстро построить модель и обучить ее, используя одну из заранее составленных архитектур или построив простую архитектуру самостоятельно. 

В рамках данного курса использования Keras вам будет достаточно для выполнения заданий.

Для начала проверим, что все необходимые библиотеки установлены. Кроме самого фреймворка, нам с вами еще понадобится пакет tensorflow_datasets для упрощения получения доступа к наборам данных для обучения.

In [25]:
!pip install numpy tensorflow tensorflow_datasets

Далее импортируем нужные пакеты

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

Кроме того, давайте посмотрим, где и как в пакете keras расположены необходимые нам элементы.
К текущему моменту мы пока что рассмотрили только один тип слоев — полносвязный, однако в дальнейшей в рамках курса мы с вами изучим и другие. Все слои располагаются в модуле layers.

In [27]:
from tensorflow.keras import layers

Кроме того, мы с вами изучили некоторые функции активации, которые можно использовать при построении ИНС. Все функции активации расположены в пакете activations.

In [28]:
from tensorflow.keras import activations

Также импортируем пакет optimizers, в котором расположены оптимизаторы для обучения.

In [29]:
from tensorflow.keras import optimizers

В модуле losses расположены различные функции потерь, которые можно использовать для обучения.

In [30]:
from tensorflow.keras import losses

И последнее, что нам понадобится импортировать — это метрики из пакета metrics. Функция потерь — это некоторая функция, которая возвращает абстрактный loss, по которому сложно или невозможно оценить качество модели. Для измерения метрик, понятных людям, воспользуемся модулем metrics и заданными там функциями.

In [31]:
from tensorflow.keras import metrics

Пакет tensorflow_datasets содержит код для скачивания открытых наборов данных. Информацию о данном пакете и список доступных наборов данных можно изучить здесь: https://www.tensorflow.org/datasets  

Данный пакет нам пригодится для того, чтобы сделать данный ноутбук воспроизводимым и не зависеть от того, правильно ли вы скачаете данные для обучения. В рамках данного урока воспользуемся набором данных <<Ирисы Фишера>>. Это простой набор данных для классификации растений, в котором используется всего 4 признака для определения одного из трех классов растений.

Загрузим упомянутый набор данных, разделив на тренировочную и тестовую выборку по границе 80% (т.е. 80% данных будет использоваться для обучения и 20% для валидации модели). Кроме того, в данном случае необходимо указать флаг as_supervised для смены формата выдачи набора данных.

In [32]:
ds_train, ds_test = tfds.load(
    name='iris',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

Давайте посмотрим на длины полученных тренировочного и валидационного набора данных и проверим, что все в порядке.

In [33]:
print(f"Lengths: {len(ds_train)}, {len(ds_test)}")

Lengths: 120, 30


Также давайте возьмем первые 5 записей из тренировочного набора данных и посмотрим на них.

In [34]:
examples = ds_train.as_numpy_iterator()
examples = [examples.next() for _ in range(5)]
features = [i[0] for i in examples]
classes = [i[1] for i in examples]

print("Features \t\t Classes")
for i in range(5):
    print(f"{features[i]} \t {classes[i]}")

Features 		 Classes
[5.1 3.4 1.5 0.2] 	 0
[7.7 3.  6.1 2.3] 	 2
[5.7 2.8 4.5 1.3] 	 1
[6.8 3.2 5.9 2.3] 	 2
[5.2 3.4 1.4 0.2] 	 0


Как можно заметить, данные представлены в следующем формате: у нас имеется 4 признака и одно поле с кодом класса. Классов всего 3. Признаки представлены в удобном для нас формате и мы можем использовать их как есть, а вот класс необходимо преобразовать с помощью one-hot encoding для дальнейшего обучения. Это необходимо нам, т.к. на последнем слое количество выходных нейронов будет равняться числу классов и мы с вами будем выбирать наиболее активировавшийся из них для определения предсказанного класса.  

Заведем константы для дальнейшего использования — размер входных данных, размер батча для обучения и количество классов для предсказания.

In [35]:
input_shape = (4, )     # Наши данные - одномерный массив с размерностью 4
batch_size = 10         # Размер батча на текущий момент можо выбрать любой, мы рассмотрим его оптимальный выбор далее в рамках курса
amount_of_classes = 3   # Количество классов определено данными - 3

Далее нам необходимо превратить один столбец с номером класса в 3 столбца, в одном из которых (соответствующему номеру класса) будет стоять единица, а в остальных — нули, т.е. провести процесс one-hot encoding'а. Для этого определим функцию, которую мы хотим применить к набору данных — функцию, которая признаки оставит нетронутыми, а колонку с классом преобразует в необходимый нам формат. Воспользуемся для этого встроенной функцией TensorFlow.

In [36]:
def make_one_hot(x, y):
    return x, tf.one_hot(y, depth = amount_of_classes)

Далее применим к тренировочному набору данных указанное преобразование, а также перемешаем его и нарежем на батчи указанного заранее нами размера. Стоит отметить флаг drop_remainder в конце — при определении батча, данный флаг поможет вам откинуть последний кусок данных, если он не совпадает по размеру с батчем, т.к. данных оказалось недостаточно (т.е. они не делятся нацело на размер батча).

In [37]:
ds_train = (
    ds_train
    .map(make_one_hot)
    .shuffle(len(ds_train))
    .batch(batch_size, drop_remainder = True)
)
    
ds_test = (
    ds_test
    .map(make_one_hot)
    .batch(batch_size, drop_remainder = True)
)

Далее определим модель. В рамках фреймворка мы с вами будем пользоваться последовательной моделью — т.е. моделью, в которой слои идут последовательно и слой связан только с предыдущим и последующим. Существуют и более сложные архитектуры, мы с вами их рассмотрим позднее.  

Создадим последовательную модель и присвоим ее в переменную

In [38]:
model = keras.Sequential()

Далее с помощью функции .add можно добавлять в нее объекты-слои. Фреймворк будет автоматически связывать их с последующими и предыдущими. Первый слой, который мы с вами добавим, это специальный InputLayer объект — слой, определяющий размер входных данных и размер пакета данных.

In [41]:
model.add(layers.InputLayer(input_shape = input_shape, batch_size = batch_size))

После этого можно добавить несколько полносвязных слоев. В каждом слое необходимо указать количество нейронов, а также функцию активации. Функцию активации можно импортировать и указать из пакета activations, которые мы импортировали ранее (заметьте, что я указываю имя функции, а не ее вызов — т.е. не указываю скобки). Также функцию активации можно указать текстовой строкой с названием функции активации (sigmoid, relu, tanh, и т.д.)

In [42]:
model.add(layers.Dense(32, activation = activations.sigmoid))
model.add(layers.Dense(16, activation = 'sigmoid'))

Последним слоем добавим слой с количеством нейронов равным количеству классов. В качестве функции активации укажем функцию softmax - это функция, реализующая функцию максимума среди всех нейронов слоя, т.е. сумма всех выходов нейронов будет составлять единицу, все выходы будут положительными, и нейрон с самой большей активацией будет иметь наиболее значение в данном промежутке - т.о. вывод данной функции можно рассматривать как приближение вероятности принадлежности экземпляра определенному классу. Данная функция, в отличии от просто функции максимума, является дифференцируемой и может быть использована для обучения методом градиентного спуска.

In [43]:
model.add(layers.Dense(amount_of_classes, activation = activations.softmax))

После определения структуры модели ее необходимо скомпилировать. Этот процесс создает граф вычислений в рамках фреймворка TensorFlow, соответствующий модели и параметрам обучения.  

В качестве параметров компиляции необходимо передать экземпляр оптимизатора — создадим для этого новый оптимизатор Adam с коэффициентом скорости обучения 0.003. Кроме того, необходимо задать функцию потерь — т.к. в нашем случае мы решаем задачу многоклассовой классификации, соответствующая функция потерь называется CategoricalCrossentropy. В качестве еще одного аргумента передадим список метрик, которые нам хотелось бы считать в процессе обучения — в данном случае укажем только точность.

In [45]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.003),
    loss = losses.CategoricalCrossentropy(),
    metrics = [metrics.CategoricalAccuracy()]
)

Процесс обучения модели запускается с помощью функции fit. В данную функцию мы подадим сам набор данных для обучения, количество эпох и набор валидационных данных. Функция запустит и обучит модель и вернет результаты в виде объекта keras.History.

In [46]:
history = model.fit(ds_train, epochs = 20, validation_data = ds_test)

Epoch 1/20
12/12 [==============================] - 1s 23ms/step - loss: 1.1161 - categorical_accuracy: 0.2833 - val_loss: 1.0982 - val_categorical_accuracy: 0.3000
Epoch 2/20
12/12 [==============================] - 0s 9ms/step - loss: 1.0988 - categorical_accuracy: 0.3083 - val_loss: 1.0942 - val_categorical_accuracy: 0.5000
Epoch 3/20
12/12 [==============================] - 0s 9ms/step - loss: 1.0994 - categorical_accuracy: 0.3583 - val_loss: 1.0944 - val_categorical_accuracy: 0.3000
Epoch 4/20
12/12 [==============================] - 0s 10ms/step - loss: 1.0866 - categorical_accuracy: 0.4333 - val_loss: 1.0816 - val_categorical_accuracy: 0.6000
Epoch 5/20
12/12 [==============================] - 0s 10ms/step - loss: 1.0799 - categorical_accuracy: 0.5917 - val_loss: 1.0680 - val_categorical_accuracy: 0.6000
Epoch 6/20
12/12 [==============================] - 0s 10ms/step - loss: 1.0569 - categorical_accuracy: 0.6500 - val_loss: 1.0463 - val_categorical_accuracy: 0.6333
Epoch 7/20
1

Данный объект (keras.History) содержит в себе историю процесса обучения, использованные параметры и ссылки на модель и объекты, использованные в процессе обучения. К примеру, можно вывести параметры обучения:

In [47]:
print(f"Params of training: {history.params}")

Params of training: {'verbose': 1, 'epochs': 20, 'steps': 12}


Также можно узнать, какой коэффициент скорости обучения использовался:

In [48]:
print(f"Used learning rate: {history.model.optimizer.lr}")

Used learning rate: <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.003>


Или даже посмотреть на веса обученного первого слоя модели:

In [49]:
print(f"Weights of first layer: {history.model.layers[0].weights}")

Weights of first layer: [<tf.Variable 'dense/kernel:0' shape=(4, 32) dtype=float32, numpy=
array([[-6.7500181e-02,  8.0909930e-02,  1.6028395e-01,  2.8937433e-02,
         3.4164953e-01, -1.7730370e-01, -3.3788927e-02, -1.2303364e-01,
        -3.3968163e-01,  3.4202796e-01, -1.0304890e-01,  9.1854535e-02,
         2.0038342e-01, -3.7304717e-01,  4.3325476e-02, -5.1417588e-03,
         5.0077688e-02, -8.2647406e-02,  4.7157648e-01, -9.5600381e-02,
        -2.7143443e-01,  3.4877896e-01,  2.9560718e-01,  3.8874272e-01,
        -2.2991458e-01,  2.0062187e-01, -3.4853894e-01, -1.7595150e-01,
         3.8466939e-01, -4.2015116e-04,  9.1322176e-02,  3.9582682e-01],
       [-5.1679927e-01,  4.5140556e-01,  1.4167824e-01, -4.9091393e-01,
         1.5929000e-01, -4.3265921e-01,  5.7061356e-01, -5.9294182e-01,
        -1.0531675e-02,  1.6056763e-01,  6.2048244e-01,  2.8663203e-01,
        -1.1149230e-03,  4.8466739e-01,  5.0815195e-01,  8.0178010e-01,
        -3.7230083e-01, -6.2644951e-02,  2.2

Далее обученную модель можно сериализовать с помощью средств фреймворка или использовать для предсказания на существующем сформированном наборе данных. Результатом предсказания будут значения трех нейронов классов, соответствующие вероятностям принадлежности экземпляра соответствующему классу.

In [50]:
model.predict(ds_test)

array([[0.9073463 , 0.07663125, 0.01602243],
       [0.9068328 , 0.07707343, 0.01609371],
       [0.00559126, 0.2246644 , 0.7697443 ],
       [0.9079279 , 0.07613076, 0.01594133],
       [0.00899539, 0.3098335 , 0.6811712 ],
       [0.00652572, 0.25007355, 0.7434007 ],
       [0.9045819 , 0.07901147, 0.01640665],
       [0.00744433, 0.2734622 , 0.7190935 ],
       [0.00660211, 0.25214058, 0.74125737],
       [0.00809676, 0.28933084, 0.70257235],
       [0.01338509, 0.3962933 , 0.5903216 ],
       [0.03343655, 0.6145688 , 0.3519946 ],
       [0.01262732, 0.38322127, 0.6041514 ],
       [0.9061336 , 0.07767531, 0.01619107],
       [0.13046868, 0.7264381 , 0.14309318],
       [0.04946885, 0.68643516, 0.26409593],
       [0.02033362, 0.49784508, 0.4818213 ],
       [0.01090373, 0.35026565, 0.6388306 ],
       [0.90698   , 0.07694661, 0.01607342],
       [0.00899735, 0.30982357, 0.6811791 ],
       [0.03642375, 0.63231266, 0.33126357],
       [0.00757621, 0.27678543, 0.71563834],
       [0.